In [5]:
# Import Modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import sqlite3
from pathlib import Path
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler

# Display all of the columns
pd.set_option('display.max_columns', None)

In [10]:
# Establish a connection to the SQLite database
conn = sqlite3.connect('resources/Poverty_DB_test02.db')

# Load data from the data_2010 to data_2022 tables
dfs = []
for year in range(2010, 2023):
    table_name = f'data_{year}'
    # Check if the table exists in the database
    query = f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'"
    table_exists = conn.execute(query).fetchone()
    if table_exists:
        query = f'SELECT * FROM {table_name}'
        df = pd.read_sql_query(query, conn)
        dfs.append(df)
# Concatenate data from all available years into a single DataFrame
data = pd.concat(dfs, ignore_index=True)
# Close the database connection
conn.close()
# Display the first few rows of the data
print(data.head())
data

   id  state_id  year  18-24_count  18-24<hs_grad  18-24_hs_grad/equiv  \
0   1         1  2010     488349.0          0.199                0.301   
1   2         2  2010      75023.0          0.188                0.371   
2   3         3  2010     634568.0          0.196                0.307   
3   4         4  2010     291054.0          0.203                0.325   
4   5         5  2010    3944934.0          0.173                0.276   

   18-24_some_college/associate  18-24_bachelor/higher   25+_count  \
0                         0.439                  0.060   3161521.0   
1                         0.407                  0.034    450577.0   
2                         0.436                  0.060   4146758.0   
3                         0.415                  0.056   1919422.0   
4                         0.473                  0.078  24097200.0   

   25+<9th_grade  25+_9th-12th_grade_nongrad  25+_hs_grad/equiv  \
0          0.062                       0.117              0.313   


In [11]:
conn = sqlite3.connect('Poverty_DB_test02.db')
states_df = pd.read_sql_query("SELECT * FROM states", conn)
states_df

DatabaseError: Execution failed on sql 'SELECT * FROM states': no such table: states

In [ ]:
# Load data from the "API_Data" table
api_data = pd.read_sql_query("SELECT * FROM API_Data", conn)

In [4]:
# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(
    Path('../resources/Poverty_DB_test02.db')
)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '..\\resources\\Poverty_DB_test02.db'

In [3]:
# Base Url for 
#url = f'https://api.census.gov/data/timeseries/poverty/saipe?get=YEAR&for=state:*&time=2018&key={api_key}'
url = f'https://api.census.gov/data/timeseries/poverty/saipe?get=NAME,SAEMHI_PT,SAEPOVRTALL_PT,SAEPOVALL_PT&YEAR&for=state:*&time=2022&key={api_key}'

In [4]:
poverty_request = requests.get(url)
poverty_request_json = poverty_request.json()

#if poverty_request.status_code == 200:
    # Printing the response content
    #print(poverty_request.json())  # Assuming the response is in JSON format
#else:
    #print("Failed to retrieve data from the API. Status code:", poverty_request.status_code)

In [5]:
pprint(poverty_request_json)

[['NAME',
  'SAEMHI_PT',
  'SAEPOVRTALL_PT',
  'SAEPOVALL_PT',
  'YEAR',
  'time',
  'state'],
 ['Alabama', '59703', '16.2', '798469', '2022', '2022', '01'],
 ['Alaska', '88072', '10.8', '77305', '2022', '2022', '02'],
 ['Arizona', '74355', '12.5', '904567', '2022', '2022', '04'],
 ['Arkansas', '55505', '16.3', '481650', '2022', '2022', '05'],
 ['California', '91517', '12.2', '4673718', '2022', '2022', '06'],
 ['Colorado', '89096', '9.5', '542333', '2022', '2022', '08'],
 ['Connecticut', '88182', '9.8', '345710', '2022', '2022', '09'],
 ['Delaware', '81933', '10.0', '99843', '2022', '2022', '10'],
 ['District of Columbia', '99897', '14.2', '91474', '2022', '2022', '11'],
 ['Florida', '69287', '12.7', '2770789', '2022', '2022', '12'],
 ['Georgia', '72742', '12.9', '1372488', '2022', '2022', '13'],
 ['Hawaii', '93683', '10.0', '140871', '2022', '2022', '15'],
 ['Idaho', '72634', '10.5', '198979', '2022', '2022', '16'],
 ['Illinois', '76744', '11.9', '1465724', '2022', '2022', '17'],
 ['I

In [4]:
dfs = []

# Loop through years 2012 to 2022 excluding 2020
for year in range(2010, 2023):
    if year != 2020:
        # Construct the API URL for the current year
        url = f'https://api.census.gov/data/timeseries/poverty/saipe?get=NAME,SAEMHI_PT,SAEPOVRTALL_PT,SAEPOVALL_PT&YEAR={year}&for=state:*&time={year}&key={api_key}'
        
        # Read data from the API into a pandas dataframe
        df = pd.read_json(url)
        
        # Remove the first row (header row) from the dataframe
        df = df.iloc[1:]
        
        # Rename the columns and set them as row headers
        df = df.rename(columns={0: 'state', 1: 'median_h_income', 2: 'poverty_rate', 3: 'poverty_count', 4: 'year', 5: 'also_year', 6: 'state_code'})
        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all dataframes into a single dataframe
result_df = pd.concat(dfs, ignore_index=True)

# Display the result
result_df.head()

,state,median_h_income,poverty_rate,poverty_count,year,also_year,state_code
0,Alabama,40538,18.9,883078,2010,2010,01
1,Alaska,63456,11.0,76850,2010,2010,02
2,Arizona,46787,17.6,1105075,2010,2010,04
3,Arkansas,38413,18.7,529710,2010,2010,05
4,California,57664,15.8,5785036,2010,2010,06


In [5]:
result_df.tail()

,state,median_h_income,poverty_rate,poverty_count,year,also_year,state_code
607,Virginia,85838,10.6,893559,2022,2022,51
608,Washington,91255,10.0,764405,2022,2022,53
609,West Virginia,54097,17.4,299275,2022,2022,54
610,Wisconsin,71050,10.8,621380,2022,2022,55
611,Wyoming,71275,11.2,63555,2022,2022,56


In [6]:
# result_df = result_df.rename(columns={'SAEMHI_PT': 'median_h_income', 'SAEPOVRTALL_PT': 'poverty_rate', 'SAEPOVALL_PT': 'poverty_count'}
#               ).drop(columns=['state', 6]
#                      )
result_df = result_df.drop(columns=['also_year', 'state_code'])

In [7]:
result_df.head()

,state,median_h_income,poverty_rate,poverty_count,year
0,Alabama,40538,18.9,883078,2010
1,Alaska,63456,11.0,76850,2010
2,Arizona,46787,17.6,1105075,2010
3,Arkansas,38413,18.7,529710,2010
4,California,57664,15.8,5785036,2010


In [9]:
#Test URL
#test_url = f'https://api.census.gov/data/timeseries/poverty/saipe?get=NAME,SAEPOVALL_PT,SAEPOVALL_MOE,SAEPOVRTALL_MOE,SAEPOVRTALL_PT&for=state:01&YEAR=2022&key={api_key}'

#test_url = "api.census.gov/data/timeseries/poverty/saipe?get=NAME,SAEPOVALL_PT,SAEPOVALL_MOE,SAEPOVRTALL_MOE,SAEPOVRTALL_PT&for=state:01&YEAR=2022"

In [10]:
#test_request = requests.get(test_url)
#test_request_json = test_request.json()

In [11]:
#pprint(test_request_json)

[['NAME',
  'SAEPOVALL_PT',
  'SAEPOVALL_MOE',
  'SAEPOVRTALL_MOE',
  'SAEPOVRTALL_PT',
  'YEAR',
  'state'],
 ['Alabama', '798469', '17536', '0.40', '16.2', '2022', '01']]


In [52]:
#This is the URL for the spread sheet and the following result from the pull
# url2 = f'https://api.census.gov/data/2022/acs/acs1/spp'
# pop_request = requests.get(url2)
# pop_request_json = pop_request.json()
# pprint(pop_request_json)

{'@context': 'https://project-open-data.cio.gov/v1.1/schema/catalog.jsonld',
 '@id': 'http://api.census.gov/data/2022/acs/acs1/spp.json',
 '@type': 'dcat:Catalog',
 'conformsTo': 'https://project-open-data.cio.gov/v1.1/schema',
 'dataset': [{'@type': 'dcat:Dataset',
              'accessLevel': 'public',
              'bureauCode': ['006:07'],
              'c_dataset': ['acs', 'acs1', 'spp'],
              'c_documentationLink': 'https://www.census.gov/developer/',
              'c_examplesLink': 'http://api.census.gov/data/2022/acs/acs1/spp/examples.json',
              'c_geographyLink': 'http://api.census.gov/data/2022/acs/acs1/spp/geography.json',
              'c_groupsLink': 'http://api.census.gov/data/2022/acs/acs1/spp/groups.json',
              'c_isAggregate': True,
              'c_isAvailable': True,
              'c_isCube': True,
              'c_sorts_url': 'http://api.census.gov/data/2022/acs/acs1/spp/sorts.json',
              'c_tagsLink': 'http://api.census.gov/data

In [3]:
# Read the all the csv data files into a Pandas DataFrame
# crowdfunding_info_df = pd.read_excel('Resources/crowdfunding.xlsx')
# crowdfunding_info_df.head()
# List of CSV file paths

directory = 'resources/'

file_paths = ['EA-2010.csv', 'EA-2011.csv', 'EA-2012.csv','EA-2013.csv','EA-2014.csv','EA-2015.csv','EA-2016.csv',
              'EA-2017.csv','EA-2018.csv','EA-2019.csv','EA-2021.csv','EA-2022.csv']

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each specified file
for file in file_paths:
    file_path = os.path.join(directory, file)
    if os.path.exists(file_path):  # Check if the file exists
        try:
            df = pd.read_csv(file_path)
            dfs.append(df)
        except Exception as e:
            print(f"Error reading file '{file_path}': {e}")
    else:
        print(f"File '{file_path}' does not exist.")

# Concatenate all DataFrames in the list into a single DataFrame
combined_csv_df = pd.concat(dfs, ignore_index=True)

In [4]:
combined_csv_df.head()

,state,year,Population 18 to 24 years,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree or higher,Population 25 years and over,Less than 9th grade,"9th to 12th grade, no diploma",...,Bachelor's degree or higher.4,MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS),Less than high school graduate.1,High school graduate (includes equivalency).2,Some college or associate's degree.1,Bachelor's degree.1,Graduate or professional degree.1,MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS),MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2021 INFLATION-ADJUSTED DOLLARS),MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)
0,Alabama,2010,"488,349",19.90%,30.10%,43.90%,6.00%,"3,161,521",6.20%,11.70%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alaska,2010,"75,023",18.80%,37.10%,40.70%,3.40%,"450,577",2.80%,6.10%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arizona,2010,"634,568",19.60%,30.70%,43.60%,6.00%,"4,146,758",6.30%,8.10%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arkansas,2010,"291,054",20.30%,32.50%,41.50%,5.60%,"1,919,422",6.30%,10.70%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,California,2010,"3,944,934",17.30%,27.60%,47.30%,7.80%,"24,097,200",10.50%,8.80%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
combined_csv_df.tail()

,state,year,Population 18 to 24 years,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree or higher,Population 25 years and over,Less than 9th grade,"9th to 12th grade, no diploma",...,Bachelor's degree or higher.4,MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS),Less than high school graduate.1,High school graduate (includes equivalency).2,Some college or associate's degree.1,Bachelor's degree.1,Graduate or professional degree.1,MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS),MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2021 INFLATION-ADJUSTED DOLLARS),MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)
595,Virginia,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"524,235",NaN,"30,436","39,730","45,135","73,893","96,914",NaN,NaN,NaN
596,Washington,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"464,118",NaN,"36,000","42,288","51,740","76,376","100,613",NaN,NaN,NaN
597,West Virginia,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"77,863",NaN,"26,692","35,052","40,791","52,887","63,974",NaN,NaN,NaN
598,Wisconsin,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"302,313",NaN,"35,193","40,104","45,726","61,428","75,811",NaN,NaN,NaN
599,Wyoming,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"28,667",NaN,"28,406","37,396","44,020","54,129","68,506",NaN,NaN,NaN


In [23]:


exdf = pd.read_csv('resources/EA-2010-copy.csv')
exdf

,state,year,Population 18 to 24 years,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree or higher,Population 25 years and over,Less than 9th grade,"9th to 12th grade, no diploma",...,Population 65 years and over,High school graduate or higher.3,Bachelor's degree or higher.4,MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2010 INFLATION-ADJUSTED DOLLARS),Population 25 years and over with earnings,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree,Graduate or professional degree
0,Alabama,2010,"488,349",19.90%,30.10%,43.90%,6.00%,"3,161,521",6.20%,11.70%,...,"657,974",72.30%,16.70%,NaN,"30,406","17,366","24,849","30,064","45,502","52,476"
1,Alaska,2010,"75,023",18.80%,37.10%,40.70%,3.40%,"450,577",2.80%,6.10%,...,"53,339",82.90%,24.40%,NaN,"40,575","20,087","30,617","40,159","51,137","67,634"
2,Arizona,2010,"634,568",19.60%,30.70%,43.60%,6.00%,"4,146,758",6.30%,8.10%,...,"886,795",83.70%,24.50%,NaN,"32,106","17,638","25,607","31,751","46,091","60,051"
3,Arkansas,2010,"291,054",20.30%,32.50%,41.50%,5.60%,"1,919,422",6.30%,10.70%,...,"420,075",72.50%,15.20%,NaN,"27,498","18,373","22,895","27,175","41,102","54,036"
4,California,2010,"3,944,934",17.30%,27.60%,47.30%,7.80%,"24,097,200",10.50%,8.80%,...,"4,269,495",76.60%,26.50%,NaN,"35,645","17,937","27,122","35,735","52,248","74,423"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Virginia,2022,"842,750","79,815","305,363","331,801","125,771","5,978,782","209,876","295,899",...,"1,462,042","1,287,105","524,235",NaN,"55,607","30,436","39,730","45,135","73,893","96,914"
596,Washington,2022,"671,550","85,472","244,136","235,801","106,141","5,470,209","192,061","236,434",...,"1,309,142","1,214,848","464,118",NaN,"59,430","36,000","42,288","51,740","76,376","100,613"
597,West Virginia,2022,"161,047","17,552","67,403","58,928","17,164","1,262,566","43,318","93,833",...,"376,162","323,520","77,863",NaN,"42,738","26,692","35,052","40,791","52,887","63,974"
598,Wisconsin,2022,"562,259","59,412","208,947","211,854","82,046","4,088,500","99,448","167,049",...,"1,102,386","1,022,332","302,313",NaN,"50,039","35,193","40,104","45,726","61,428","75,811"


: 

In [21]:

import os

# Assuming your CSV files are stored in a directory named 'csv_files'
directory = 'resources'

# List all CSV files in the directory
csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

# Create an empty list to store DataFrames
dfs = []

# Read each CSV file into a DataFrame and append it to the list
for file in csv_files:
    df = pd.read_csv(os.path.join(directory, file))
    if not dfs:  # If dfs is empty, just append the DataFrame without filtering columns
        dfs.append(df)
    else:
        # Select only the columns that exist in the first DataFrame
        df = df[df.columns.intersection(dfs[0].columns)]
        dfs.append(df)

# Concatenate all DataFrames into one big DataFrame
big_df = pd.concat(dfs, ignore_index=True)

# Sort the big DataFrame by year
big_df = big_df.sort_values(by='year')

# Reset index
big_df.reset_index(drop=True, inplace=True)

# Print the concatenated DataFrame
print(big_df)
big_df

              state  year Population 18 to 24 years  \
0          Alabama   2010                   488,349   
1           Nevada   2010                   250,339   
2    New Hampshire   2010                   123,240   
3       New Jersey   2010                   767,908   
4       New Mexico   2010                   205,957   
..              ...   ...                       ...   
595       Maryland   2022                       NaN   
596  Massachusetts   2022                       NaN   
597       Michigan   2022                       NaN   
598         Hawaii   2022                       NaN   
599        Wyoming   2022                       NaN   

    Less than high school graduate  \
0                           19.90%   
1                           21.90%   
2                           13.20%   
3                           13.30%   
4                           22.30%   
..                             ...   
595                            NaN   
596                            NaN 

,state,year,Population 18 to 24 years,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree or higher,Population 25 years and over,Less than 9th grade,"9th to 12th grade, no diploma",...,Population 65 years and over,High school graduate or higher.3,Bachelor's degree or higher.4,MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2010 INFLATION-ADJUSTED DOLLARS),Population 25 years and over with earnings,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree,Graduate or professional degree
0,Alabama,2010,"488,349",19.90%,30.10%,43.90%,6.00%,"3,161,521",6.20%,11.70%,...,"657,974",72.30%,16.70%,NaN,"30,406","17,366","24,849","30,064","45,502","52,476"
1,Nevada,2010,"250,339",21.90%,36.10%,37.00%,5.00%,"1,789,530",6.30%,9.10%,...,"327,133",82.20%,20.00%,NaN,"31,973","22,101","28,844","33,812","42,498","64,184"
2,New Hampshire,2010,"123,240",13.20%,30.60%,45.50%,10.70%,"907,755",2.80%,5.80%,...,"179,649",83.30%,24.50%,NaN,"38,050","22,637","31,204","35,772","46,991","63,294"
3,New Jersey,2010,"767,908",13.30%,30.30%,43.70%,12.70%,"5,970,904",5.40%,6.50%,...,"1,191,030",78.00%,23.20%,NaN,"42,473","21,463","31,295","40,213","57,081","80,417"
4,New Mexico,2010,"205,957",22.30%,29.20%,43.00%,5.50%,"1,341,329",7.60%,9.10%,...,"275,387",77.00%,24.50%,NaN,"30,125","16,636","24,540","30,101","41,639","51,771"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Maryland,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"61,166",NaN,NaN,NaN,NaN,NaN
596,Massachusetts,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"62,009",NaN,NaN,NaN,NaN,NaN
597,Michigan,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"47,259",NaN,NaN,NaN,NaN,NaN
598,Hawaii,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"50,993",NaN,NaN,NaN,NaN,NaN


In [22]:
big_df.columns

Index(['state', 'year', 'Population 18 to 24 years',
       'Less than high school graduate',
       'High school graduate (includes equivalency)',
       'Some college or associate's degree', 'Bachelor's degree or higher',
       'Population 25 years and over', 'Less than 9th grade',
       '9th to 12th grade, no diploma',
       'High school graduate (includes equivalency).1',
       'Some college, no degree', 'Associate's degree', 'Bachelor's degree',
       'Graduate or professional degree', 'Population 25 to 34 years',
       'High school graduate or higher', 'Bachelor's degree or higher.1',
       'Population 35 to 44 years', 'High school graduate or higher.1',
       'Bachelor's degree or higher.2', 'Population 45 to 64 years',
       'High school graduate or higher.2', 'Bachelor's degree or higher.3',
       'Population 65 years and over', 'High school graduate or higher.3',
       'Bachelor's degree or higher.4',
       'MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2010 INFLATION